In [7]:
#Import Libraries
import numpy as np
import pandas as pd
import requests
import random
import time
from io import StringIO

In [8]:
#Create list of team codes 
teams = [
'ATL', 'BOS', 'BRK', 'CHA', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 
'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 
'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS'
]
len(teams)

30

In [9]:
#Create code: team name dictionary
team_dict = {
    'ATL': 'Hawks', 'BOS': 'Celtics', 'BRK': 'Nets', 'CHA': 'Hornets', 'CHI': 'Bulls', 'CLE': 'Cavaliers',
    'DAL': 'Mavericks', 'DEN': 'Nuggets', 'DET': 'Pistons', 'GSW': 'Warriors', 'HOU': 'Rockets', 'IND': 'Pacers', 
    'LAC': 'Clippers', 'LAL': 'Lakers', 'MEM': 'Grizzlies', 'MIA': 'Heat', 'MIL': 'Bucks', 'MIN': 'Timberwolves', 
    'NOP': 'Pelicans', 'NYK': 'Knicks', 'OKC': 'Thunder', 'ORL': 'Magic', 'PHI': '76ers', 'PHO': 'Suns', 
    'POR': 'Trail Blazers', 'SAC': 'Kings', 'SAS': 'Spurs', 'TOR': 'Raptors', 'UTA': 'Jazz', 'WAS': 'Wizards'
}
len(team_dict)

30

In [10]:
#Create the list of years (seasons)
seasons = ['2022', '2023']
len(seasons)

2

In [11]:
#Create an empty dataframe
nba_df = pd.DataFrame()

#Iterate thru seasons
for season in seasons:

    #Iterate thru teams
    for team in teams:

        #SET URL
        url = 'https://www.basketball-reference.com/teams/' + team + '/'+ season + '/gamelog-advanced/'
        print(url)

        #Get a response object
        response = requests.get(url ,headers={'User-agent' : 'Mozilla/5.0'})

        visible_html = response.text.replace('<!--' , '')

        #Get dataframes
        df_list = pd.read_html(StringIO(visible_html), header=1)
        if len(df_list) == 1:
            team_df = df_list[0]
        else:
            df_list[1]['G'] = df_list[1]['G'] + 82
            team_df = pd.concat([df_list[0], df_list[1]], ignore_index=True)

            #Drop rows where 'G' is Nan or 'G"
            team_df = team_df.dropna(subset=['G'])
            team_df = team_df[team_df['G'] != 'G']
            team_df = team_df.reset_index(drop=True)

            #Keeping Columns 1-11
            team_df = team_df.iloc[:, 1:11]

            #Rename some colums
            team_df = team_df.rename(columns={'Unnamed: 3': 'At', 'Tm':'OPts', 'Opp.1':'DPts'})

            #Replace values in columns Home and Opp of teamdf
            team_df['At'] = team_df['At'].apply(lambda x: x if x== '@' else "")
            team_df['Win'] = team_df['W/L'].apply(lambda x: 1 if x== 'W' else 0)
            team_df = team_df.drop(columns=['W/L'])

            #Adding columns
            team_df.insert(loc=0, column='Season', value=season)
            team_df.insert(loc=2, column='Team', value=team)
            team_df.insert(loc=3, column='Name', value=team_dict[team])
            team_df['Location'] = team_df['At'].apply(lambda x: 'Away' if x=='@' else 'Home')
            team_df.insert(loc=6, column='Location', value=team_df.pop('Location'))
            team_df['Opp_Name'] = team_df['Opp'].apply(lambda x: team_dict[x])
            team_df.insert(loc=8, column='Opp_Name', value=team_df.pop('Opp_Name'))
            #print(team_df)

            #Aggregate the dataframe
            nba_df = pd.concat([nba_df, team_df], ignore_index=True)

            #Pause to abide by bball-ref.com rule
            time.sleep(random.randin(4, 6))
#display the dataframe
print(nba_df.shape)

https://www.basketball-reference.com/teams/ATL/2022/gamelog-advanced/


KeyError: 'PHO'

In [ ]:
print(nba_df)

In [ ]:
#Update Datatypes
nba_df['G'] = nba_df['G'].astype('int')
nba_df['OPts'] = nba_df['OPts'].astype('int')
nba_df['DPts'] = nba_df['DPts'].astype('int')
nba_df['ORtg'] = nba_df['ORtg'].astype('float')
nba_df['DRtg'] = nba_df['DRtg'].astype('float')
nba_df['Pace'] = nba_df['Pace'].astype('float')

In [ ]:
print(nba_df.info())
print(nba_df.head())

In [ ]:
#SAVE TO CSV
nba_df.to_csv('nba-advanced-gamelogs-22-23-TEST.csv', index=False)